Copie de Calendrier

In [9]:
import pandas as pd
chemin1 = '/workspaces/metro-JO/plages_hor_debut_event.csv'
chemin2 = "/workspaces/metro-JO/plages_hor_fin_event.csv"
new_horaires_début_df= pd.read_csv(chemin1, delimiter=';')
new_horaires_fin_df= pd.read_csv(chemin2, delimiter=';')

In [10]:
import pandas as pd
import tabula
import numpy as np

# %%
lieux = [
'stade-tour-eiffel',
'arena-champ-de-mars',
'grand-palais',
'invalides',
'pont-alexandre-iii',
'trocadero', 
'concorde',
'stade-roland-garros',
'arena-paris-sud', 
'arena-bercy', 
'arena-la-chapelle', 
'paris-la-defense-arena', 
'centre-aquatique', 
'stade-de-france',
'site-escalade-bourget',
'arena-paris-nord'
]

vecteur = [lieux[0] if i < 4 else
           lieux[1] if i < 8 else
           lieux[2] if i < 13 else
           lieux[3] if i < 15 else
           lieux[4] if i < 17 else
           lieux[5] if i < 19 else
           lieux[6] if i < 29 else
           lieux[7] if i < 35 else
           lieux[8] if i < 48 else
           lieux[9] if i < 58 else
           lieux[10] if i < 63 else
           lieux[11] if i < 68 else
           lieux[12] if i < 74 else
           lieux[13] if i < 78 else
           lieux[14] if i < 79 else
           lieux[15] if i < 83 else
           'Valeur par défaut' for i in range(83)]


# Lien vers le PDF contenant le tableau
lien_pdf = "https://medias.paris2024.org/uploads/2022/07/Calendrier-par-epreuves-des-Jeux-Olympiques-de-Paris-2024.pdf"

try : 
    # Utilisation de tabula pour extraire le tableau du PDF
    # Utilisation de l'option 'pages' pour spécifier la page ou les pages contenant le tableau
    df_list = tabula.read_pdf(lien_pdf, pages=[1, 2], stream=True)  # Adapter les options selon vos besoins

    # Concaténer les DataFrames obtenus des différentes pages en un seul DataFrame
    df = pd.concat(df_list)

    # Supprimer les 28 dernières lignes
    df = df.iloc[:-32]
    df.drop(24, inplace=True)
    df.drop(23, inplace=True)
    df.drop(55, inplace=True)

    # Supprimer la première ligne du DataFrame
    df = df.drop(df.index[[0, 2]])  # Suppression de la première ligne
    df = df.drop(df.columns[[0]], axis=1)

    # Supprimer les lignes contenant uniquement des NaN
    df = df.dropna(how='all')
    
    df.columns = df.iloc[0]
    df.drop(1, inplace=True)

    df.index = vecteur
    
    df = df.replace('<', np.nan)

except Exception as e:
        print("Une erreur s'est produite :", str(e))




# %%
def format_date_horaire(cell, index_value):
    if isinstance(cell, str) and '-' in cell:
        date_month = '07' if int(index_value) in range(24, 32) else '08'
        year = '2024'
        
        date = f"{index_value}/{date_month}/{year}"
        horaire1, horaire2 = cell.split('-')
        
        return f"{date} {horaire1.strip()} - {horaire2.strip()}"
    else:
        return cell

# Appliquer la fonction à chaque cellule du DataFrame
for column in df.columns:
    df[column] = df.apply(lambda x: format_date_horaire(x[column], column), axis=1)


# %%
# Supprimer les titres des index
df.index.name = None

# Ajouter Lieux pour la première colonne
df.reset_index(inplace=True)
df.columns = ['Lieux'] + list(df.columns[1:])  # Renommer la première colonne en 'Lieux'

# %%
# Créer un dictionnaire pour stocker les valeurs non vides associées à chaque lieu
data = {}
for idx, row in df.iterrows():
    lieu = row['Lieux']
    if lieu not in data:
        data[lieu] = []

    # Récupérer les valeurs non vides et les ajouter à la liste des valeurs du lieu
    values = row.drop('Lieux').dropna().tolist()
    data[lieu].extend(values)

# Créer un nouveau DataFrame à partir du dictionnaire
new_df = pd.DataFrame.from_dict(data, orient='index')

lieux_a_supprimer = ['arena-paris-nord', 'site-escalade-bourget', 'centre-aquatique']

# Suppression des lignes spécifiées
new_df = new_df.drop(lieux_a_supprimer, axis=0)

# %%
# Fonction pour convertir le format de date
def convert_date_format(entry):
    if isinstance(entry, str):
        parts = entry.split(' ')
        if len(parts) == 5:
            date = parts[0]
            start_time = parts[1]
            return f"{date} {start_time}"
    return entry

# Appliquer la fonction sur toutes les colonnes du DataFrame
for col in new_df.columns:
    new_df[col] = new_df[col].apply(convert_date_format)


# %%
# Appliquer une fonction pour supprimer la partie après le tiret dans chaque cellule
horaires_début = new_df.applymap(lambda x: x.split(' -')[0] if isinstance(x, str) else x)

# Appliquer une fonction pour supprimer les 4 caractères avant le tiret dans chaque cellule
horaires_fin = new_df.applymap(lambda x: pd.Series(x).replace(r'.{5}\s-\s', '', regex=True)[0] if isinstance(x, str) else x)

horaires_début = horaires_début.applymap(lambda x: pd.to_datetime(x, errors='coerce'))
horaires_début = horaires_début.dropna(axis=1, how='all')

horaires_fin = horaires_fin.applymap(lambda x: pd.to_datetime(x, errors='coerce'))
horaires_fin = horaires_fin.dropna(axis=1, how='all')

# Afficher le DataFrame résultant
display(horaires_début)
display(horaires_fin)


# %%
def adjust_time_moins_90min(cell):
    # Vérifier si la cellule est vide ou non
    if not pd.isnull(cell):
        # Convertir la cellule en objet datetime
        cell = pd.to_datetime(cell, format='%d/%m/%Y %H:%M')
        # Ajouter 1 heure et 30 minutes à l'objet datetime
        cell = cell - pd.Timedelta(hours=1, minutes=30)
    return cell

def adjust_time_plus_30min(cell):
    # Vérifier si la cellule est vide ou non
    if not pd.isnull(cell):
        # Convertir la cellule en objet datetime
        cell = pd.to_datetime(cell, format='%d/%m/%Y %H:%M')
        # Retirer 30 minutes à l'objet datetime
        cell = cell + pd.Timedelta(hours=0, minutes=30)
    return cell

# Parcourir chaque cellule du DataFrame et appliquer la fonction adjust_time
new_horaires_début_df = horaires_début.map(adjust_time_moins_90min)
new_horaires_fin_df = horaires_fin.map(adjust_time_plus_30min)

new_horaires_début_df.reset_index(inplace=True)
new_horaires_début_df.rename(columns={'index': 'Lieu'}, inplace=True)

new_horaires_fin_df.reset_index(inplace=True)
new_horaires_fin_df.rename(columns={'index': 'Lieu'}, inplace=True)

display(new_horaires_début_df)
display(new_horaires_fin_df)

new_horaires_fin_df.to_excel('new_horaires_fin_df.xlsx', index=False)
new_horaires_début_df.to_excel('new_horaires_début_df.xlsx', index=False)

# %%
def ajuster_plage_horaire(heure):
    if pd.notnull(heure):
        # Extraire la date et l'heure de la cellule
        date, time = str(heure).split(' ')
        heure, minute, seconde = map(int, time.split(':')[0:3])
        
        # Calculer la plage horaire correspondante
        plage_horaire = f"{heure:02d}:00:00 - {(heure+1)%24:02d}:00:00"
        
        # Construire la date avec la plage horaire
        return f"{date} {plage_horaire}"
    
    return heure

# Appliquer la fonction à toutes les colonnes (sauf la première)
new_horaires_début_df.iloc[:, 1:] = new_horaires_début_df.iloc[:, 1:].map(ajuster_plage_horaire)
new_horaires_fin_df.iloc[:, 1:] = new_horaires_fin_df.iloc[:, 1:].map(ajuster_plage_horaire)

display(new_horaires_début_df)
display(new_horaires_fin_df)


# %% [markdown]
# Ajout du script Nb de personnes + Arrêts + Lignes des lieux des événements

# %%
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

# %% [markdown]
# Liste des lieux et url 

# %%

lieux = [
'stade-tour-eiffel',
'arena-champ-de-mars',
'grand-palais',
'invalides',
'pont-alexandre-iii',
'trocadero', 
'concorde',
'stade-roland-garros',
'arena-paris-sud', 
'arena-bercy', 
'arena-la-chapelle', 
'paris-la-defense-arena', 
'centre-aquatique', 
'stade-de-france',
'site-escalade-bourget',
'arena-paris-nord'
]

urls = [
    f"https://www.paris2024.org/fr/site/{lieu.replace(' ', '-').lower()}/" for lieu in lieux
]

# %%
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_specific_lines(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        specific_lines = {
            'Capacité': [],
            'Stations': []  # Création d'une liste pour stocker les stations
        }
        for paragraph in soup.find_all('p'):
            # Recherche des lignes contenant les Capacité spécifiques
            if ('Capacité' in paragraph.text or 
                'Capacité court Philippe-Chatrier' in paragraph.text or 
                'Volleyball' in paragraph.text or 
                'Pour la gymnastique rythmique' in paragraph.text):
                specific_lines['Capacité'].append(paragraph.text.strip())
                
            # Extraction des phrases après "desservie" ou "desservi"
            if 'desservie' in paragraph.text or 'desservi' in paragraph.text:
                sentences = paragraph.text.split('.')  # Séparation en phrases
                for sentence in sentences:
                    if 'desservie' in sentence or 'desservi' in sentence:
                        specific_lines['Stations'].append(sentence.strip())
        
        return specific_lines
    return None


# Création d'un dictionnaire pour stocker les résultats
results = {lieu: get_specific_lines(url) for lieu, url in zip(lieux, urls)}

# Création d'un DataFrame à partir du dictionnaire
df = pd.DataFrame(results.items(), columns=['Lieu', 'Infos'])
# Expansion du dictionnaire pour créer les colonnes 'Capacité' et 'Stations'
df = pd.concat([df.drop(['Infos'], axis=1), df['Infos'].apply(pd.Series)], axis=1)


# %% [markdown]
# Compléter pour les données manquantes : Concorde 

# %%
# Faites une requête GET pour récupérer le contenu HTML de la page
url = 'https://sportetsociete.org/2022/10/20/paris-2024-un-cadre-majestueux-pour-louverture-des-jeux-paralympiques/#:~:text=Des%20tribunes%20de%2035%20000,des%20Champs%2DÉlysées%20pour%20saluer'
response = requests.get(url)

# Vérifiez si la requête a réussi
if response.status_code == 200:
    # Utilisation de BeautifulSoup pour analyser le contenu HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Recherchez le texte contenant "places"
    texte_places = soup.find(text=lambda text: text and "places" in text)

    if texte_places:
        # Imprimez le texte trouvé
        df.iloc[6, 1]=texte_places.strip()

# %% [markdown]
# Compléter pour les données manquantes : Arena Paris Sud

# %%
#Compléter pour les valeurs manquantes : arena-paris-sud
url = "https://fr.wikipedia.org/wiki/Arena_Porte_de_la_Chapelle"

# Obtenir le contenu de la page
response = requests.get(url)

if response.status_code == 200:
    # Analyser le contenu HTML avec BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Trouver le texte de la ligne commençant par "Capacité"
    capacity_text = soup.find("th", text="Capacité").find_next("td").get_text(strip=True)

    # Mettre à jour la case correspondante dans le DataFrame df
    df.iloc[8, 1] = capacity_text  # Ligne 8, colonne 2, indexées à partir de 0

else:
    print("La requête a échoué")

# %%
def extraire_nombres(texte):
    nombres = re.findall(r'[-+]?\d*\.\d+|\d+', str(texte))  # Expression régulière pour les nombres entiers ou décimaux
    return ' '.join(nombres)  # Retourne les nombres extraits sous forme d'une chaîne séparée par des espaces

# Appliquer la fonction sur la colonne 'Capacité'
df['Nombre de personnes à l\'événément'] = df['Capacité'].apply(extraire_nombres)

# Supposons que 'df' est votre DataFrame contenant la colonne 'Nombres'

# Remplacer "202" par une chaîne vide dans la colonne 'Nombre de personnes à l\'événément'
df['Nombre de personnes à l\'événément'] = df['Nombre de personnes à l\'événément'].str.replace('202', '')

# Supprimer les espaces dans la colonne 'NNombre de personnes à l\'événément'
df['Nombre de personnes à l\'événément'] = df['Nombre de personnes à l\'événément'].str.replace(' ', '')

# Supposons que 'df' est votre DataFrame contenant la colonne 'Nombres'

# Remplacer les valeurs dans la colonne 'Nombres' aux lignes spécifiques
df.loc[2, 'Nombre de personnes à l\'événément'] = '8000'  # Remplacer la deuxième ligne par '8000'
df.loc[10, 'Nombre de personnes à l\'événément'] = '6700'  # Remplacer la 10ème ligne par '6700'
df.loc[14, 'Nombre de personnes à l\'événément'] = '6000'  # Remplacer la 14ème ligne par '6000'

# Fonction pour supprimer le zéro en tête des nombres
def remove_leading_zero(number):
    if number.startswith('0'):
        return number[1:]
    return number

# Appliquer la fonction à la colonne 'Nombres' pour supprimer les zéros en tête
df['Nombre de personnes à l\'événément'] = df['Nombre de personnes à l\'événément'].astype(str).apply(remove_leading_zero)


# %%
# Boucle pour lire chaque ligne de la colonne 'Stations'
for index, station in df['Stations'].items():
    print(station)

# %%
arrêts=[['BIR HAKEIM', 'ALMA MARCEAU'],
        ['LA MOTTE-PICQUET-GRENELLE', 'LA MOTTE-PICQUET-GRENELLE', 'SEGUR'], 
        ['FRANKLIN D. ROOSEVELT', 'FRANKLIN D. ROOSEVELT'], 
        ['INVALIDES'], 
        ['INVALIDES'],
        ['IENA','TROCADERO'], 
        ['PALAIS-ROYAL','PALAIS-ROYAL','MADELEINE', 'MADELEINE', 'MADELEINE', 'OPERA'],
        ['PORTE DE SAINT-CLOUD','PORTE D\'AUTEUIL'], 
        ['PORTE DE VERSAILLES','BALARD','PORTE DE VANVES'], 
        ['BERCY','BERCY','GARE DE LYON'],
        ['PORTE DE LA CHAPELLE'],
        ['LA DEFENSE'],
        ['x'],
        ['SAINT-DENIS-PORTE DE PARIS'], 
        ['x'],
        ['x']]
 
df['Arrêts']=arrêts

# Fonction pour extraire les nombres après le mot "Métro"
def extract_numbers(text):
    matches = re.findall(r'Métro\s+(\d+)', str(text), flags=re.IGNORECASE)
    return matches

# Appliquer la fonction à la colonne 'Stations' pour créer une nouvelle colonne 'Nombres après Métro'
df['Lignes'] = df['Stations'].apply(extract_numbers)

# Compléter pour Franklin D
df.at[2, 'Lignes'] = [6, 9]

# %%
# Supprimer les colonnes 'Capacité' et 'Stations'
colonnes_a_supprimer = ['Capacité', 'Stations']
df = df.drop(colonnes_a_supprimer, axis=1)

# Calculer le nombre d'éléments dans chaque vecteur de la ligne 'Lignes'
df['Nombre de lignes'] = df['Lignes'].apply(lambda x: len(x) if isinstance(x, list) else None)

# Convertir la colonne 'Nombre de personnes à l\'événément' en type numérique (int ou float)
df['Nombre de personnes à l\'événément'] = pd.to_numeric(df['Nombre de personnes à l\'événément'], errors='coerce')

# Créer la nouvelle colonne en calculant la division
df['Nombre de personnes par ligne'] = df['Nombre de personnes à l\'événément'] / df['Nombre de lignes']

# Définir l'option d'affichage pour éviter la notation scientifique
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# %%
# Identifier les indices des lignes contenant les valeurs spécifiques dans la colonne 'lieu'
indices_a_supprimer = df[df['Lieu'].isin(['centre-aquatique', 'site-escalade-bourget', 'arena-paris-nord'])].index

# Supprimer les lignes correspondantes
df = df.drop(indices_a_supprimer)

display(df)

df.to_excel('df.xlsx', index=False)

# %% [markdown]
# Copie du programme fréquentation_metro

# %%
# Copie du programme Nettoyage données affluence par jour + nombre de validation par jour par station

# %%
import pandas as pd
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url = '/workspaces/metro-JO/Données importées/2022_S2_PROFIL_FER.csv'
données_validation = pd.read_csv(url, delimiter=';')

# %%
# Convertir la colonne 'pourc_validations' en float (remplacer la virgule par un point pour séparer les décimales)
données_validation['pourc_validations'] = données_validation['pourc_validations'].str.replace(',', '.').astype(float)

# Grouper par 'LIBELLE_ARRET' et 'TRNC_HORR_60', puis calculer la somme des pourc_validations pour chaque groupe
tableau_somme_pour_validations = données_validation.groupby(['LIBELLE_ARRET', 'TRNC_HORR_60'])['pourc_validations'].sum().reset_index()


# %%
# Mapping des tranches horaires à des heures spécifiques
tranche_horaire_to_hour = {
    '0H-1H': '00:00-01:00',
    '1H-2H': '01:00-02:00',
    '2H-3H': '02:00-03:00',
    '3H-4H': '03:00-04:00',
    '4H-5H': '04:00-05:00',
    '5H-6H': '05:00-06:00',
    '6H-7H': '06:00-07:00',
    '7H-8H': '07:00-08:00',
    '8H-9H': '08:00-09:00',
    '9H-10H': '09:00-10:00',
    '10H-11H': '10:00-11:00',
    '11H-12H': '11:00-12:00',
    '12H-13H': '12:00-13:00',
    '13H-14H': '13:00-14:00',
    '14H-15H': '14:00-15:00',
    '15H-16H': '15:00-16:00',
    '16H-17H': '16:00-17:00',
    '17H-18H': '17:00-18:00',
    '18H-19H': '18:00-19:00',
    '19H-20H': '19:00-20:00',
    '20H-21H': '20:00-21:00',
    '21H-22H': '21:00-22:00',
    '22H-23H': '22:00-23:00',
    '23H-24H': '23:00-00:00'
}

# Appliquer la conversion des tranches horaires au format heure
tableau_somme_pour_validations['TRNC_HORR_60'] = tableau_somme_pour_validations['TRNC_HORR_60'].map(tranche_horaire_to_hour)

# %%
# Regroupement des données en cas de doublons pour une paire 'LIBELLE_ARRET'-'TRNC_HORR_60'
grouped_data = données_validation.groupby(['LIBELLE_ARRET', 'TRNC_HORR_60'])['pourc_validations'].sum().reset_index()

# Pivotement des données
pivot_table = grouped_data.pivot(index='LIBELLE_ARRET', columns='TRNC_HORR_60', values='pourc_validations').reset_index()

# Renommage des colonnes
pivot_table.columns.name = None  # Supprimer le nom de la colonne index
pivot_table = pivot_table.rename(columns=lambda x: f"{x}" if x != 'LIBELLE_ARRET' else x)

# %%
import re

# Sélectionner les colonnes correspondant aux tranches horaires
horaire_columns = pivot_table.columns[1:]  # Exclure la colonne 'LIBELLE_ARRET'

# Fonction pour extraire l'heure de début de chaque tranche horaire
def extract_start_hour(column):
    match = re.match(r'(\d+)H-(\d+)H', column)
    if match:
        return int(match.group(1))
    return 0

# Trier les colonnes dans l'ordre croissant des heures de début des tranches
sorted_columns = sorted(horaire_columns, key=extract_start_hour)

# Réorganiser le tableau pivoté avec les colonnes dans l'ordre chronologique
new_column_order = ['LIBELLE_ARRET'] + sorted_columns
pivot_table = pivot_table[new_column_order]

# Supprimer la colonne 'ND' du DataFrame pivot_table_ordered
pivot_table.drop(columns=['ND'], inplace=True, errors='ignore')

pivot_table['LIBELLE_ARRET'] = pivot_table['LIBELLE_ARRET'].str.strip()  # Supprime les espaces en début et fin
pivot_table['LIBELLE_ARRET'] = pivot_table['LIBELLE_ARRET'].str.lower()  # Met tout en minuscules

# Créer un dictionnaire de correspondance pour renommer les colonnes
correspondance_heures = {
    '0H-1H': '00:00-01:00', '1H-2H': '01:00-02:00', '2H-3H': '02:00-03:00', '3H-4H': '03:00-04:00',
    '4H-5H': '04:00-05:00', '5H-6H': '05:00-06:00', '6H-7H': '06:00-07:00', '7H-8H': '07:00-08:00',
    '8H-9H': '08:00-09:00', '9H-10H': '09:00-10:00', '10H-11H': '10:00-11:00', '11H-12H': '11:00-12:00',
    '12H-13H': '12:00-13:00', '13H-14H': '13:00-14:00', '14H-15H': '14:00-15:00', '15H-16H': '15:00-16:00',
    '16H-17H': '16:00-17:00', '17H-18H': '17:00-18:00', '18H-19H': '18:00-19:00', '19H-20H': '19:00-20:00',
    '20H-21H': '20:00-21:00', '21H-22H': '21:00-22:00', '22H-23H': '22:00-23:00', '23H-0H': '23:00-00:00'
}

# Renommer les colonnes en utilisant le dictionnaire de correspondance
pivot_table.rename(columns=correspondance_heures, inplace=True)

# Afficher le DataFrame après suppression de la colonne 'ND'
print(pivot_table)

ModuleNotFoundError: No module named 'tabula'

Copie de Nb de personnes + Arrêts + Lignes des lieux des événements

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

# %% [markdown]
# Liste des lieux et url 

# %%

lieux = [
'stade-tour-eiffel',
'arena-champ-de-mars',
'grand-palais',
'invalides',
'pont-alexandre-iii',
'trocadero', 
'concorde',
'stade-roland-garros',
'arena-paris-sud', 
'arena-bercy', 
'arena-la-chapelle', 
'paris-la-defense-arena', 
'centre-aquatique', 
'stade-de-france',
'site-escalade-bourget',
'arena-paris-nord'
]

urls = [
    f"https://www.paris2024.org/fr/site/{lieu.replace(' ', '-').lower()}/" for lieu in lieux
]

# %%
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_specific_lines(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        specific_lines = {
            'Capacité': [],
            'Stations': []  # Création d'une liste pour stocker les stations
        }
        for paragraph in soup.find_all('p'):
            # Recherche des lignes contenant les Capacité spécifiques
            if ('Capacité' in paragraph.text or 
                'Capacité court Philippe-Chatrier' in paragraph.text or 
                'Volleyball' in paragraph.text or 
                'Pour la gymnastique rythmique' in paragraph.text):
                specific_lines['Capacité'].append(paragraph.text.strip())
                
            # Extraction des phrases après "desservie" ou "desservi"
            if 'desservie' in paragraph.text or 'desservi' in paragraph.text:
                sentences = paragraph.text.split('.')  # Séparation en phrases
                for sentence in sentences:
                    if 'desservie' in sentence or 'desservi' in sentence:
                        specific_lines['Stations'].append(sentence.strip())
        
        return specific_lines
    return None


# Création d'un dictionnaire pour stocker les résultats
results = {lieu: get_specific_lines(url) for lieu, url in zip(lieux, urls)}

# Création d'un DataFrame à partir du dictionnaire
df = pd.DataFrame(results.items(), columns=['Lieu', 'Infos'])
# Expansion du dictionnaire pour créer les colonnes 'Capacité' et 'Stations'
df = pd.concat([df.drop(['Infos'], axis=1), df['Infos'].apply(pd.Series)], axis=1)


# %% [markdown]
# Compléter pour les données manquantes : Concorde 

# %%
# Faites une requête GET pour récupérer le contenu HTML de la page
url = 'https://sportetsociete.org/2022/10/20/paris-2024-un-cadre-majestueux-pour-louverture-des-jeux-paralympiques/#:~:text=Des%20tribunes%20de%2035%20000,des%20Champs%2DÉlysées%20pour%20saluer'
response = requests.get(url)

# Vérifiez si la requête a réussi
if response.status_code == 200:
    # Utilisation de BeautifulSoup pour analyser le contenu HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Recherchez le texte contenant "places"
    texte_places = soup.find(text=lambda text: text and "places" in text)

    if texte_places:
        # Imprimez le texte trouvé
        df.iloc[6, 1]=texte_places.strip()

# %% [markdown]
# Compléter pour les données manquantes : Arena Paris Sud

# %%
#Compléter pour les valeurs manquantes : arena-paris-sud
url = "https://fr.wikipedia.org/wiki/Arena_Porte_de_la_Chapelle"

# Obtenir le contenu de la page
response = requests.get(url)

if response.status_code == 200:
    # Analyser le contenu HTML avec BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Trouver le texte de la ligne commençant par "Capacité"
    capacity_text = soup.find("th", text="Capacité").find_next("td").get_text(strip=True)

    # Mettre à jour la case correspondante dans le DataFrame df
    df.iloc[8, 1] = capacity_text  # Ligne 8, colonne 2, indexées à partir de 0

else:
    print("La requête a échoué")

# %%
def extraire_nombres(texte):
    nombres = re.findall(r'[-+]?\d*\.\d+|\d+', str(texte))  # Expression régulière pour les nombres entiers ou décimaux
    return ' '.join(nombres)  # Retourne les nombres extraits sous forme d'une chaîne séparée par des espaces

# Appliquer la fonction sur la colonne 'Capacité'
df['Nombre de personnes à l\'événément'] = df['Capacité'].apply(extraire_nombres)

# Supposons que 'df' est votre DataFrame contenant la colonne 'Nombres'

# Remplacer "202" par une chaîne vide dans la colonne 'Nombre de personnes à l\'événément'
df['Nombre de personnes à l\'événément'] = df['Nombre de personnes à l\'événément'].str.replace('202', '')

# Supprimer les espaces dans la colonne 'NNombre de personnes à l\'événément'
df['Nombre de personnes à l\'événément'] = df['Nombre de personnes à l\'événément'].str.replace(' ', '')

# Supposons que 'df' est votre DataFrame contenant la colonne 'Nombres'

# Remplacer les valeurs dans la colonne 'Nombres' aux lignes spécifiques
df.loc[2, 'Nombre de personnes à l\'événément'] = '8000'  # Remplacer la deuxième ligne par '8000'
df.loc[10, 'Nombre de personnes à l\'événément'] = '6700'  # Remplacer la 10ème ligne par '6700'
df.loc[14, 'Nombre de personnes à l\'événément'] = '6000'  # Remplacer la 14ème ligne par '6000'

# Fonction pour supprimer le zéro en tête des nombres
def remove_leading_zero(number):
    if number.startswith('0'):
        return number[1:]
    return number

# Appliquer la fonction à la colonne 'Nombres' pour supprimer les zéros en tête
df['Nombre de personnes à l\'événément'] = df['Nombre de personnes à l\'événément'].astype(str).apply(remove_leading_zero)

# %%
arrêts=[['BIR HAKEIM', 'ALMA MARCEAU'],
        ['LA MOTTE-PICQUET-GRENELLE', 'LA MOTTE-PICQUET-GRENELLE', 'SEGUR'], 
        ['FRANKLIN D. ROOSEVELT', 'FRANKLIN D. ROOSEVELT'], 
        ['INVALIDES'], 
        ['INVALIDES'],
        ['IENA','TROCADERO'], 
        ['PALAIS-ROYAL','PALAIS-ROYAL','MADELEINE', 'MADELEINE', 'MADELEINE', 'OPERA'],
        ['PORTE DE SAINT-CLOUD','PORTE D\'AUTEUIL'], 
        ['PORTE DE VERSAILLES','BALARD','PORTE DE VANVES'], 
        ['BERCY','BERCY','GARE DE LYON'],
        ['PORTE DE LA CHAPELLE'],
        ['LA DEFENSE'],
        ['x'],
        ['SAINT-DENIS-PORTE DE PARIS'], 
        ['x'],
        ['x']]
 
df['Arrêts']=arrêts

# Fonction pour extraire les nombres après le mot "Métro"
def extract_numbers(text):
    matches = re.findall(r'Métro\s+(\d+)', str(text), flags=re.IGNORECASE)
    return matches

# Appliquer la fonction à la colonne 'Stations' pour créer une nouvelle colonne 'Nombres après Métro'
df['Lignes'] = df['Stations'].apply(extract_numbers)

# Compléter pour Franklin D
df.at[2, 'Lignes'] = [6, 9]

# %%
# Supprimer les colonnes 'Capacité' et 'Stations'
colonnes_a_supprimer = ['Capacité', 'Stations']
df = df.drop(colonnes_a_supprimer, axis=1)

# Calculer le nombre d'éléments dans chaque vecteur de la ligne 'Lignes'
df['Nombre de lignes'] = df['Lignes'].apply(lambda x: len(x) if isinstance(x, list) else None)

# Convertir la colonne 'Nombre de personnes à l\'événément' en type numérique (int ou float)
df['Nombre de personnes à l\'événément'] = pd.to_numeric(df['Nombre de personnes à l\'événément'], errors='coerce')

# Créer la nouvelle colonne en calculant la division
df['Nombre de personnes par ligne'] = df['Nombre de personnes à l\'événément'] / df['Nombre de lignes']

# Définir l'option d'affichage pour éviter la notation scientifique
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# %%
# Identifier les indices des lignes contenant les valeurs spécifiques dans la colonne 'lieu'
indices_a_supprimer = df[df['Lieu'].isin(['centre-aquatique', 'site-escalade-bourget', 'arena-paris-nord'])].index

# Supprimer les lignes correspondantes
df = df.drop(indices_a_supprimer)

display(df)

df.to_excel('df.xlsx', index=False)

/tmp/ipykernel_1597/864975592.py:89: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  texte_places = soup.find(text=lambda text: text and "places" in text)
/tmp/ipykernel_1597/864975592.py:110: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  capacity_text = soup.find("th", text="Capacité").find_next("td").get_text(strip=True)


,Lieu,Nombre de personnes à l'événément,Arrêts,Lignes,Nombre de lignes,Nombre de personnes par ligne
0,stade-tour-eiffel,12860,"[BIR HAKEIM, ALMA MARCEAU]","[6, 9]",2,6430.00
1,arena-champ-de-mars,8356,"[LA MOTTE-PICQUET-GRENELLE, LA MOTTE-PICQUET-G...","[6, 8, 10]",3,2785.33
2,grand-palais,8000,"[FRANKLIN D. ROOSEVELT, FRANKLIN D. ROOSEVELT]","[6, 9]",2,4000.00
3,invalides,8000,[INVALIDES],"[8, 13]",2,4000.00
4,pont-alexandre-iii,1000,[INVALIDES],"[8, 13]",2,500.00
5,trocadero,3349,"[IENA, TROCADERO]","[9, 6]",2,1674.50
6,concorde,35000,"[PALAIS-ROYAL, PALAIS-ROYAL, MADELEINE, MADELE...","[1, 7, 8, 12, 14, 3]",6,5833.33
7,stade-roland-garros,14962,"[PORTE DE SAINT-CLOUD, PORTE D'AUTEUIL]","[9, 10]",2,7481.00
8,arena-paris-sud,9000,"[PORTE DE VERSAILLES, BALARD, PORTE DE VANVES]","[12, 8, 13]",3,3000.00
9,arena-bercy,15000,"[BERCY, BERCY, GARE DE LYON]","[6, 14, 1]",3,5000.00


Copie du programme fréquentation_metro

In [ ]:
# %%
import pandas as pd
import numpy as np
import re
import difflib

# %% [markdown]
# Copie du programme Nettoyage données affluence par jour + nombre de validation par jour par station

# %%
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url = '/workspaces/metro-JO/Données importées/2022_S2_PROFIL_FER.csv'
données_validation = pd.read_csv(url, delimiter=';')

# Convertir la colonne 'pourc_validations' en float (remplacer la virgule par un point pour séparer les décimales)
données_validation['pourc_validations'] = données_validation['pourc_validations'].str.replace(',', '.').astype(float)

# Grouper par 'LIBELLE_ARRET' et 'TRNC_HORR_60', puis calculer la somme des pourc_validations pour chaque groupe
tableau_somme_pour_validations = données_validation.groupby(['LIBELLE_ARRET', 'TRNC_HORR_60'])['pourc_validations'].sum().reset_index()


# Mapping des tranches horaires à des heures spécifiques
tranche_horaire_to_hour = {
    '0H-1H': '00:00-01:00',
    '1H-2H': '01:00-02:00',
    '2H-3H': '02:00-03:00',
    '3H-4H': '03:00-04:00',
    '4H-5H': '04:00-05:00',
    '5H-6H': '05:00-06:00',
    '6H-7H': '06:00-07:00',
    '7H-8H': '07:00-08:00',
    '8H-9H': '08:00-09:00',
    '9H-10H': '09:00-10:00',
    '10H-11H': '10:00-11:00',
    '11H-12H': '11:00-12:00',
    '12H-13H': '12:00-13:00',
    '13H-14H': '13:00-14:00',
    '14H-15H': '14:00-15:00',
    '15H-16H': '15:00-16:00',
    '16H-17H': '16:00-17:00',
    '17H-18H': '17:00-18:00',
    '18H-19H': '18:00-19:00',
    '19H-20H': '19:00-20:00',
    '20H-21H': '20:00-21:00',
    '21H-22H': '21:00-22:00',
    '22H-23H': '22:00-23:00',
    '23H-24H': '23:00-00:00'
}

# Appliquer la conversion des tranches horaires au format heure
tableau_somme_pour_validations['TRNC_HORR_60'] = tableau_somme_pour_validations['TRNC_HORR_60'].map(tranche_horaire_to_hour)

# Regroupement des données en cas de doublons pour une paire 'LIBELLE_ARRET'-'TRNC_HORR_60'
grouped_data = données_validation.groupby(['LIBELLE_ARRET', 'TRNC_HORR_60'])['pourc_validations'].sum().reset_index()

# Pivotement des données
pivot_table = grouped_data.pivot(index='LIBELLE_ARRET', columns='TRNC_HORR_60', values='pourc_validations').reset_index()

# Renommage des colonnes
pivot_table.columns.name = None  # Supprimer le nom de la colonne index
pivot_table = pivot_table.rename(columns=lambda x: f"{x}" if x != 'LIBELLE_ARRET' else x)

# Sélectionner les colonnes correspondant aux tranches horaires
horaire_columns = pivot_table.columns[1:]  # Exclure la colonne 'LIBELLE_ARRET'

# Fonction pour extraire l'heure de début de chaque tranche horaire
def extract_start_hour(column):
    match = re.match(r'(\d+)H-(\d+)H', column)
    if match:
        return int(match.group(1))
    return 0

# Trier les colonnes dans l'ordre croissant des heures de début des tranches
sorted_columns = sorted(horaire_columns, key=extract_start_hour)

# Réorganiser le tableau pivoté avec les colonnes dans l'ordre chronologique
new_column_order = ['LIBELLE_ARRET'] + sorted_columns
pivot_table = pivot_table[new_column_order]

# Supprimer la colonne 'ND' du DataFrame pivot_table_ordered
pivot_table.drop(columns=['ND'], inplace=True, errors='ignore')

pivot_table['LIBELLE_ARRET'] = pivot_table['LIBELLE_ARRET'].str.strip()  # Supprime les espaces en début et fin
pivot_table['LIBELLE_ARRET'] = pivot_table['LIBELLE_ARRET'].str.lower()  # Met tout en minuscules

# Créer un dictionnaire de correspondance pour renommer les colonnes
correspondance_heures = {
    '0H-1H': '00:00-01:00', '1H-2H': '01:00-02:00', '2H-3H': '02:00-03:00', '3H-4H': '03:00-04:00',
    '4H-5H': '04:00-05:00', '5H-6H': '05:00-06:00', '6H-7H': '06:00-07:00', '7H-8H': '07:00-08:00',
    '8H-9H': '08:00-09:00', '9H-10H': '09:00-10:00', '10H-11H': '10:00-11:00', '11H-12H': '11:00-12:00',
    '12H-13H': '12:00-13:00', '13H-14H': '13:00-14:00', '14H-15H': '14:00-15:00', '15H-16H': '15:00-16:00',
    '16H-17H': '16:00-17:00', '17H-18H': '17:00-18:00', '18H-19H': '18:00-19:00', '19H-20H': '19:00-20:00',
    '20H-21H': '20:00-21:00', '21H-22H': '21:00-22:00', '22H-23H': '22:00-23:00', '23H-0H': '23:00-00:00'
}

# Renommer les colonnes en utilisant le dictionnaire de correspondance
pivot_table.rename(columns=correspondance_heures, inplace=True)

# Afficher le DataFrame après suppression de la colonne 'ND'
print(pivot_table)

# %% [markdown]
# Tableau avec station, ligne, correspondances...

# %%
tableau_fréquentation = pd.read_excel('https://data.ratp.fr/api/explore/v2.1/catalog/datasets/trafic-annuel-entrant-par-station-du-reseau-ferre-2021/exports/xlsx?lang=fr&timezone=Europe%2FBerlin&use_labels=true')

print(tableau_fréquentation)

# %%
tableau_fréquentation = tableau_fréquentation[tableau_fréquentation['Réseau'] == 'Métro']
display(tableau_fréquentation)

# %%
colonnes_a_supprimer = ['Réseau', 'Ville', 'Arrondissement pour Paris']

tableau_fréquentation = tableau_fréquentation.drop(colonnes_a_supprimer, axis=1)

# %%
# Liste des colonnes à modifier
colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

# Convertir les colonnes spécifiées en type chaîne (si elles ne le sont pas déjà)
tableau_fréquentation[colonnes_correspondance] = tableau_fréquentation[colonnes_correspondance].astype(str)

# Supprimer les lettres 'A' et 'B' des colonnes spécifiées dans tableau_fréquentation
for colonne in colonnes_correspondance:
    tableau_fréquentation[colonne] = tableau_fréquentation[colonne].str.replace('A', '').str.replace('B', '')

# %% [markdown]
# PROBLÈME À RÉGLER : CREER UNE COLONNE QUI COMPTE LE NOMBRE DE LIGNES

# %%
# Liste des colonnes à analyser
colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

# Calcul du nombre de cases non vides pour chaque ligne
tableau_fréquentation['nb_correspondance'] = tableau_fréquentation[colonnes_correspondance].count(axis=1)

# %%
# Diviser les valeurs de la colonne "Trafic" par les valeurs de la colonne "nb_correspondance"
tableau_fréquentation['Trafic'] = tableau_fréquentation['Trafic'] / tableau_fréquentation['nb_correspondance']
print(tableau_fréquentation)

# %% [markdown]
# Création d'un tableau avec la moyenne sur la période des JO en 2022 du nombre de validation hebdomadire à chaque station

# %%
#Nettoyage de données sur le nombre de validations: on obtient le nombre de validation moyen par jour sur 1 mois précis

import pandas as pd
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url2 = '/workspaces/metro-JO/Données importées/2022_S2_NB_FER.csv'
données_validation2= pd.read_csv(url2)
données_validation2 = pd.read_csv(url2, delimiter=';')

# Exemple avec gestion de formats flexibles
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], errors='coerce', format="%d/%m/%Y")
donnees_mai = données_validation2.loc[(données_validation2['JOUR'].dt.month == 5)]
donnees_sum = donnees_mai.groupby('LIBELLE_ARRET')['NB_VALD'].mean().reset_index()

# Conversion de la colonne 'JOUR' en datetime
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], format='%d/%m/%Y')

# Filtrer les données pour les dates spécifiques
donnees_filtrees = données_validation2[(données_validation2['JOUR'] >= '2022-07-24') & (données_validation2['JOUR'] <= '2022-08-11')]

# Grouper par 'JOUR', 'LIBELLE_ARRET' et 'CATEGORIE_TITRE' et calculer la somme des 'NB_VALD'
donnees_aggreg = donnees_filtrees.groupby(['JOUR', 'LIBELLE_ARRET', 'CATEGORIE_TITRE'])['NB_VALD'].sum().reset_index()

# Conversion de la colonne 'JOUR' en datetime si ce n'est pas déjà fait
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], format='%d/%m/%Y')

# Filtrage des dates spécifiques
donnees_filtrees = données_validation2[(données_validation2['JOUR'] >= '2022-07-24') & (données_validation2['JOUR'] <= '2022-08-11')]

# Somme de NB_VALD par JOUR et par LIBELLE_ARRET
somme_par_jour_arret = donnees_filtrees.groupby(['JOUR', 'LIBELLE_ARRET'])['NB_VALD'].sum().reset_index()

moyenne_par_arret = somme_par_jour_arret.groupby('LIBELLE_ARRET')['NB_VALD'].mean().reset_index()

# Normalisation des libellés
moyenne_par_arret['LIBELLE_ARRET'] = moyenne_par_arret['LIBELLE_ARRET'].str.lower().str.strip()

# Somme des valeurs de NB_VALD pour les libellés normalisés
moyenne_par_arret = moyenne_par_arret.groupby('LIBELLE_ARRET')['NB_VALD'].sum().reset_index()

# Suppression des doublons
moyenne_par_arret = moyenne_par_arret.drop_duplicates(subset=['LIBELLE_ARRET'])

display(moyenne_par_arret)

# %% [markdown]
# Création d'un dictionnaire contenant les fréquentations de chaque station de chaque ligne --> PROBLÈME À RÉGLER : CERTAINES CASES VIDES

# %%
numéro = [1, 2, 3, '3bis', 4, 5, 6, 7, '7bis', 8, 9, 10, 11, 12, 13, 14]

# Création des tableaux pour chaque élément de numéro
tableaux = {}

colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

for k in numéro:
    # Convertir k en string car il peut contenir des valeurs non numériques
    k_str = str(k)
    
    # Filtrer les lignes où la valeur k est présente dans les colonnes spécifiées
    filtered_rows = tableau_fréquentation[tableau_fréquentation[colonnes_correspondance].apply(lambda x: k_str in x.values, axis=1)]
    
    # Stocker les lignes filtrées dans un tableau nommé "ligne numéro[k]"
    tableaux[f'ligne numéro[{k_str}]'] = filtered_rows

# Parcourir chaque tableau dans le dictionnaire et supprimer les colonnes spécifiées
for tableau_nom, tableau in tableaux.items():
    tableaux[tableau_nom] = tableau.drop(columns=colonnes_correspondance)

# Parcourir chaque tableau dans le dictionnaire
for tableau_nom, tableau in tableaux.items():
    # Calculer la fréquentation totale pour chaque tableau
    fréquentation_totale_ligne = tableau['Trafic'].sum()

    # Ajouter la variable fréquentation_totale à chaque tableau
    tableaux[tableau_nom]['fréquentation_totale_ligne'] = fréquentation_totale_ligne
    tableau['Part du trafic de la station par rapport à toute la ligne'] = tableau['Trafic'] / tableau['fréquentation_totale_ligne']

# Supposons que la clé pour la correspondance soit 'LIBELLE_ARRET' dans le tableau moyenne_par_arret
cle_correspondance = 'LIBELLE_ARRET'

# Nettoyage des espaces et uniformisation de la casse
moyenne_par_arret['LIBELLE_ARRET'] = moyenne_par_arret['LIBELLE_ARRET'].str.strip().str.lower()
for tableau_nom, tableau in tableaux.items():
    tableau['Station'] = tableau['Station'].str.strip().str.lower()

    # Fusion en utilisant les noms de stations nettoyés
    merged_table = pd.merge(tableau, moyenne_par_arret, left_on='Station', right_on='LIBELLE_ARRET', how='left')
    
    # Renommer la colonne NB_VALD de moyenne_par_arret
    merged_table = merged_table.rename(columns={'NB_VALD': 'fréquentation moyenne hors JO'})
    
    tableaux[tableau_nom] = merged_table

# %%
#compléter avec les valeurs manquantes

# Méthode pour trouver la correspondance la plus proche
def find_closest_match(target, candidates):
    closest_match = difflib.get_close_matches(target, candidates, n=1, cutoff=0.4)  # Ajustez le seuil selon vos besoins
    return closest_match[0] if closest_match else None

for k in numéro:
    k_str = str(k)
    # Accéder aux tableaux spécifiques
    current_table = tableaux[f'ligne numéro[{k_str}]']
    
    # Filtrer les lignes avec des valeurs manquantes dans la colonne 'LIBELLE_ARRET'
    missing_data = current_table[current_table['LIBELLE_ARRET'].isnull()]
    
    # Remplir les valeurs manquantes
    for index, row in missing_data.iterrows():
        closest_match = find_closest_match(row['Station'], moyenne_par_arret['LIBELLE_ARRET'])
        
        if closest_match:
            tableaux[f'ligne numéro[{k_str}]'].at[index, 'LIBELLE_ARRET'] = closest_match



# %%
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Fusionner avec le tableau moyenne_par_arret pour obtenir les valeurs NB_VALD associées
    merged_with_nb_vald = pd.merge(current_table, moyenne_par_arret[['LIBELLE_ARRET', 'NB_VALD']],
                                   left_on='LIBELLE_ARRET', right_on='LIBELLE_ARRET', how='left')
    
    # Renommer la colonne NB_VALD pour plus de clarté
    merged_with_nb_vald = merged_with_nb_vald.rename(columns={'NB_VALD': 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'})

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = merged_with_nb_vald

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Fusionner les données avec le tableau contenant les nouvelles colonnes
    merged_table = pd.merge(current_table, pivot_table, on='LIBELLE_ARRET', how='left')

    # Mettre à jour le tableau dans le dictionnaire avec les nouvelles colonnes fusionnées
    tableaux[f'ligne numéro[{k_str}]'] = merged_table

# %%
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Supprimer les colonnes 'fréquentation moyenne hors JO' et 'LIBELLE_ARRET' si elles existent
    columns_to_drop = ['fréquentation moyenne hors JO', 'LIBELLE_ARRET']
    current_table = current_table.drop(columns=columns_to_drop, errors='ignore')

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = current_table



# %%
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si les colonnes existent avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station' in current_table.columns and 'nb_correspondance' in current_table.columns:
        # Calcul de la nouvelle colonne
        current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'] / current_table['nb_correspondance']

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si les colonnes existent avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station' in current_table.columns and 'nb_correspondance' in current_table.columns:
        # Calcul de la nouvelle colonne
        current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'] / current_table['nb_correspondance'] / 2

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si la colonne existe avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne' in current_table.columns:
        # Calculer la somme de la colonne spécifique
        current_table['nombre_validations_totales_pour_la_ligne'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne'].sum()

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table
        # Afficher le tableau mis à jour
        print(f"Tableau 'ligne numéro[{k_str}]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :")
        display(tableaux[f'ligne numéro[{k_str}]'])

/tmp/ipykernel_1597/2055730461.py:13: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  données_validation = pd.read_csv(url, delimiter=';')


       LIBELLE_ARRET  00:00-01:00  01:00-02:00  02:00-03:00  03:00-04:00  \
0           abbesses        10.18         3.20         0.02         0.00   
1              ablon         0.26         0.03         0.00         0.02   
2      acheres grand         0.78          NaN          NaN          NaN   
3      acheres ville         0.76         0.31         0.13         0.09   
4      aeroport cdg1         0.76         0.14         0.01         0.04   
..               ...          ...          ...          ...          ...   
751           vosves          NaN          NaN          NaN          NaN   
752  vulaines sur se         0.11          NaN          NaN          NaN   
753           wagram         3.63         1.43         0.10         0.00   
754           yerres         0.73         0.11         0.01         0.05   
755   yvris noisy gd         0.20         0.04          NaN          NaN   

     04:00-05:00  05:00-06:00  06:00-07:00  07:00-08:00  08:00-09:00  ...  \
0         

,Rang,Réseau,Station,Trafic,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,Ville,Arrondissement pour Paris
0,3,Métro,GARE DE LYON,28640475,1,14,A,NaN,NaN,Paris,12.00
1,12,Métro,BELLEVILLE,7314438,2,11,NaN,NaN,NaN,Paris,10.00
2,32,Métro,OPERA,5193831,3,7,8,NaN,NaN,Paris,9.00
3,58,Métro,SAINT-MANDE-TOURELLE,3944640,1,NaN,NaN,NaN,NaN,Saint Mandé,NaN
4,59,Métro,HOCHE,3928404,5,NaN,NaN,NaN,NaN,Pantin,NaN
...,...,...,...,...,...,...,...,...,...,...,...
350,272,Métro,SULLY-MORLAND,1124169,7,NaN,NaN,NaN,NaN,Paris,4.00
351,275,Métro,EUROPE,1106459,3,NaN,NaN,NaN,NaN,Paris,8.00
352,283,Métro,CAMPO-FORMIO,1016150,5,NaN,NaN,NaN,NaN,Paris,13.00
353,298,Métro,SAINT-FARGEAU,472258,3bis,NaN,NaN,NaN,NaN,Paris,8.00


     Rang               Station     Trafic Correspondance_1 Correspondance_2  \
0       3          GARE DE LYON 5728095.00                1               14   
1      12            BELLEVILLE 1462887.60                2               11   
2      32                 OPERA 1038766.20                3                7   
3      58  SAINT-MANDE-TOURELLE  788928.00                1              nan   
4      59                 HOCHE  785680.80                5              nan   
..    ...                   ...        ...              ...              ...   
350   272         SULLY-MORLAND  224833.80                7              nan   
351   275                EUROPE  221291.80                3              nan   
352   283          CAMPO-FORMIO  203230.00                5              nan   
353   298         SAINT-FARGEAU   94451.60             3bis              nan   
354   300       PORTE D'AUTEUIL   75149.60               10              nan   

    Correspondance_3 Correspondance_4 C

,LIBELLE_ARRET,NB_VALD
0,abbesses,2277.32
1,ablon,1101.11
2,acheres grand,51.39
3,acheres ville,3012.05
4,aeroport cdg1,5050.95
...,...,...
733,vosves,14.89
734,vulaines sur se,42.89
735,wagram,3268.16
736,yerres,2743.00


...

,Rang,Station,Trafic,nb_correspondance,fréquentation_totale_ligne,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,242,rue du bac,284672.80,5,23051055.60,0.01,2642.84,7.98,2.21,0.04,...,57.77,57.82,48.87,23.13,15.46,12.09,11.93,528.57,264.28,105667.29
1,288,rennes,172266.80,5,23051055.60,0.01,1546.05,4.45,1.57,0.08,...,58.20,56.48,44.05,23.63,10.44,10.24,8.72,309.21,154.61,105667.29
2,4,montparnasse-bienvenue,4081444.80,5,23051055.60,0.18,67112.68,5.71,0.26,0.01,...,55.50,62.72,49.65,31.05,21.54,16.83,13.49,13422.54,6711.27,105667.29
3,4,montparnasse-bienvenue,4081444.80,5,23051055.60,0.18,67112.68,8.99,1.58,0.08,...,40.52,44.32,41.17,32.13,24.22,21.47,19.70,13422.54,6711.27,105667.29
4,27,madeleine,1066185.60,5,23051055.60,0.05,12904.42,4.97,1.52,0.11,...,65.73,71.49,60.81,23.76,11.64,11.65,10.10,2580.88,1290.44,105667.29
5,88,sevres-babylone,678500.80,5,23051055.60,0.03,6540.11,4.71,1.28,0.07,...,59.22,66.54,57.85,35.27,12.25,9.71,8.96,1308.02,654.01,105667.29
6,115,mairie d'issy,574827.60,5,23051055.60,0.02,6616.53,2.55,0.89,0.06,...,38.97,38.91,30.74,16.37,8.48,6.66,5.20,1323.31,661.65,105667.29
7,188,notre-dame-de-lorette,378765.60,5,23051055.60,0.02,3949.58,10.62,4.42,0.04,...,44.18,49.69,39.57,25.65,17.51,18.02,18.33,789.92,394.96,105667.29
8,93,porte de versailles,653631.40,5,23051055.60,0.03,15023.95,12.16,4.87,0.09,...,60.08,62.00,53.19,32.02,19.18,19.01,18.49,3004.79,1502.39,105667.29
9,136,vaugirard,496577.20,5,23051055.60,0.02,4768.74,4.38,1.34,0.07,...,44.22,41.04,34.89,19.16,10.52,9.18,7.66,953.75,476.87,105667.29


Tableau 'ligne numéro[13]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :


,Rang,Station,Trafic,nb_correspondance,fréquentation_totale_ligne,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,74,saint-denis-universite,713998.00,5,16957848.60,0.04,8049.37,1.95,0.42,0.01,...,31.51,27.77,18.64,13.60,8.89,6.17,4.36,1609.87,804.94,37710.67
1,122,les courtilles,560537.80,5,16957848.60,0.03,8152.79,2.39,0.58,0.02,...,35.24,31.76,23.70,16.18,10.77,7.34,4.88,1630.56,815.28,37710.67
2,125,porte de saint-ouen,542127.60,5,16957848.60,0.03,6262.37,5.48,2.05,0.05,...,42.89,40.38,34.53,23.47,16.62,12.24,9.27,1252.47,626.24,37710.67
3,29,porte de clichy,1055699.40,5,16957848.60,0.06,12615.11,9.95,3.26,0.27,...,92.43,88.85,71.86,46.94,32.62,24.38,18.66,2523.02,1261.51,37710.67
4,33,place clichy,1032386.40,5,16957848.60,0.06,10922.74,13.84,5.13,0.13,...,41.68,40.48,38.04,26.06,20.67,21.62,21.26,2184.55,1092.27,37710.67
5,87,porte de vanves,679071.60,5,16957848.60,0.04,7711.32,4.61,1.30,0.07,...,41.18,39.19,30.95,21.34,13.95,10.18,8.11,1542.26,771.13,37710.67
6,162,pernety,434713.40,5,16957848.60,0.03,4404.26,5.99,1.92,0.09,...,34.66,34.29,31.53,21.05,13.67,12.71,10.96,880.85,440.43,37710.67
7,230,carrefour pleyel,307105.60,5,16957848.60,0.02,3933.79,3.40,1.46,0.10,...,57.68,45.30,28.87,16.22,10.19,8.39,6.78,786.76,393.38,37710.67
8,64,gabriel peri,758114.60,5,16957848.60,0.04,9837.84,3.24,0.79,0.02,...,38.88,38.49,29.48,18.40,11.34,7.23,5.40,1967.57,983.78,37710.67
9,69,miromesnil,729451.00,5,16957848.60,0.04,8603.79,8.08,2.75,0.05,...,60.25,65.41,47.45,26.32,13.69,14.06,11.90,1720.76,860.38,37710.67


Tableau 'ligne numéro[14]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :


,Rang,Station,Trafic,nb_correspondance,fréquentation_totale_ligne,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,3,gare de lyon,5728095.00,5,15764240.20,0.36,66037.95,2.16,0.01,0.00,...,57.99,60.21,38.73,27.60,17.40,10.24,4.80,13207.59,6603.79,47415.80
1,3,gare de lyon,5728095.00,5,15764240.20,0.36,66037.95,18.38,4.31,0.30,...,84.89,89.26,74.77,56.04,41.78,38.76,34.89,13207.59,6603.79,47415.80
2,61,bercy,776842.40,5,15764240.20,0.05,9023.95,NaN,NaN,NaN,...,104.32,67.23,40.18,13.57,0.11,14.23,0.09,1804.79,902.39,47415.80
3,61,bercy,776842.40,5,15764240.20,0.05,9023.95,10.59,2.39,0.22,...,43.36,44.68,36.17,35.22,24.11,25.58,36.50,1804.79,902.39,47415.80
4,27,madeleine,1066185.60,5,15764240.20,0.07,12904.42,4.97,1.52,0.11,...,65.73,71.49,60.81,23.76,11.64,11.65,10.10,2580.88,1290.44,47415.80
5,29,porte de clichy,1055699.40,5,15764240.20,0.07,12615.11,9.95,3.26,0.27,...,92.43,88.85,71.86,46.94,32.62,24.38,18.66,2523.02,1261.51,47415.80
6,110,cour saint-emilion,597024.40,5,15764240.20,0.04,3173.44,11.21,1.93,0.12,...,61.54,60.91,45.62,29.16,25.69,15.63,12.76,634.69,317.34,47415.80
7,6,bibliotheque,2220894.80,5,15764240.20,0.14,6088.00,6.46,1.35,0.06,...,54.02,56.83,44.83,26.99,18.84,11.58,8.76,1217.60,608.80,47415.80
8,31,olympiades,1042919.00,5,15764240.20,0.07,12997.14,2.74,1.23,0.06,...,47.26,41.98,31.78,20.50,13.19,6.90,4.47,2599.43,1299.71,47415.80
9,39,mairie de saint-ouen,966162.00,5,15764240.20,0.06,12017.74,7.75,2.71,0.19,...,93.80,89.63,63.45,40.80,24.30,16.35,13.66,2403.55,1201.77,47415.80


In [ ]:
def calculer_personnes_supplementaires(date_horaire_entree, numero_ligne, station):

    tableau_debut = pd.DataFrame()  # Initialisation du DataFrame vide
    tableau_fin = pd.DataFrame()    # Initialisation du DataFrame vide

    # Extraire l'horaire de l'entrée utilisateur
    heure_entree = date_horaire_entree[-5:]

    # Déterminer la plage horaire à laquelle appartient l'horaire d'entrée
    plage_horaire_entree = f"{heure_entree.split(':')[0]}:00:00 - {(int(heure_entree.split(':')[0])+1)%24:02d}:00:00"

    # Construire la chaîne de date avec la plage horaire
    date_plage_horaire = f"{date_horaire_entree[:10]} {plage_horaire_entree}"

    # Vérifier si la chaîne date_plage_horaire se trouve dans new_horaires_début_df
    if (date_plage_horaire in new_horaires_début_df.values) or (date_plage_horaire in new_horaires_fin_df.values): 
        # Afficher les éléments de la colonne "Lieu" correspondants à la condition dans new_horaires_début_df
        lieux_correspondants_debut = new_horaires_début_df.loc[new_horaires_début_df.eq(date_plage_horaire).any(axis=1), 'Lieu']
        # Afficher les éléments de la colonne "Lieu" correspondants à la condition dans new_horaires_fin_df
        lieux_correspondants_fin = new_horaires_fin_df.loc[new_horaires_fin_df.eq(date_plage_horaire).any(axis=1), 'Lieu']
        
        lignes_correspondantes = []
        
        
        # Renvoyer les vecteurs de la colonne Ligne pour chaque élément de lieux_correspondants_debut
        for lieu in lieux_correspondants_debut:
            ligne_lieu = df[df['Lieu'] == lieu]  # Trouver la ligne correspondant au lieu
            tableau_debut=pd.concat([tableau_debut, ligne_lieu], ignore_index=True)
            lignes_lieu = ligne_lieu['Lignes']
            lignes_correspondantes.extend(lignes_lieu.values)
            

        # Renvoyer les vecteurs de la colonne Ligne pour chaque élément de lieux_correspondants_fin
        for lieu in lieux_correspondants_fin:
            ligne_lieu = df[df['Lieu'] == lieu]  # Trouver la ligne correspondant au lieu
            tableau_fin=pd.concat([tableau_fin, ligne_lieu], ignore_index=True)
            lignes_lieu = ligne_lieu['Lignes']
            lignes_correspondantes.extend(lignes_lieu.values)

        numero_trouve = False

        for sous_liste in lignes_correspondantes:
            if numero_ligne in sous_liste:
                numero_trouve = True
                break  # Arrête la boucle une fois que le numéro est trouvé

            # Vérifier si le numéro de ligne entré est présent dans les vecteurs des lignes correspondantes
        if numero_trouve:
            resultat_debut=0
            resultat_fin=0
            if not tableau_debut.empty :
                tableau_debut = tableau_debut[tableau_debut['Lignes'].apply(lambda x: numero_ligne in x)]
                resultat_debut = tableau_debut['Nombre de personnes par ligne'].sum()
            if not tableau_fin.empty :
                    tableau_fin = tableau_fin[tableau_fin['Lignes'].apply(lambda x: numero_ligne in x)]
                    resultat_fin = tableau_fin['Nombre de personnes par ligne'].sum()
            resultat=resultat_debut+resultat_fin
            print(f"Le nombre de personnes supplémentaires sur la ligne en raison des JO  est {resultat}")
                    

        else:
            print(0)
    else:
        print(0)

    display(tableau_debut)
    display(tableau_fin)

calculer_personnes_supplementaires('2024-07-28 09:50', '1', 'la-defense')

Le nombre de personnes supplémentaires sur la ligne en raison des JO  est 17000.0


,Lieu,Nombre de personnes à l'événément,Arrêts,Lignes,Nombre de lignes,Nombre de personnes par ligne
0,paris-la-defense-arena,17000,[LA DEFENSE],[1],1,17000.00


""


Programme

In [ ]:
def calculer_personnes_supplementaires(date_horaire_entree, numero_ligne, station):

    tableau_debut = pd.DataFrame()  # Initialisation du DataFrame vide
    tableau_fin = pd.DataFrame()    # Initialisation du DataFrame vide

    cle_tableau = f'ligne numéro[{numero_ligne}]'

    # Extraire l'horaire de l'entrée utilisateur
    heure_entree = date_horaire_entree[-5:]

    # Déterminer la plage horaire à laquelle appartient l'horaire d'entrée
    plage_horaire_entree = f"{heure_entree.split(':')[0]}:00:00 - {(int(heure_entree.split(':')[0])+1)%24:02d}:00:00"

    # Construire la chaîne de date avec la plage horaire
    date_plage_horaire = f"{date_horaire_entree[:10]} {plage_horaire_entree}"

    # Vérifier si la chaîne date_plage_horaire se trouve dans new_horaires_début_df
    if (date_plage_horaire in new_horaires_début_df.values) or (date_plage_horaire in new_horaires_fin_df.values): 
        # Afficher les éléments de la colonne "Lieu" correspondants à la condition dans new_horaires_début_df
        lieux_correspondants_debut = new_horaires_début_df.loc[new_horaires_début_df.eq(date_plage_horaire).any(axis=1), 'Lieu']
        # Afficher les éléments de la colonne "Lieu" correspondants à la condition dans new_horaires_fin_df
        lieux_correspondants_fin = new_horaires_fin_df.loc[new_horaires_fin_df.eq(date_plage_horaire).any(axis=1), 'Lieu']
        
        lignes_correspondantes = []
        
        
        # Renvoyer les vecteurs de la colonne Ligne pour chaque élément de lieux_correspondants_debut
        for lieu in lieux_correspondants_debut:
            ligne_lieu = df[df['Lieu'] == lieu]  # Trouver la ligne correspondant au lieu
            tableau_debut=pd.concat([tableau_debut, ligne_lieu], ignore_index=True)
            lignes_lieu = ligne_lieu['Lignes']
            lignes_correspondantes.extend(lignes_lieu.values)
            

        # Renvoyer les vecteurs de la colonne Ligne pour chaque élément de lieux_correspondants_fin
        for lieu in lieux_correspondants_fin:
            ligne_lieu = df[df['Lieu'] == lieu]  # Trouver la ligne correspondant au lieu
            tableau_fin=pd.concat([tableau_fin, ligne_lieu], ignore_index=True)
            lignes_lieu = ligne_lieu['Lignes']
            lignes_correspondantes.extend(lignes_lieu.values)

        numero_trouve = False

        for sous_liste in lignes_correspondantes:
            if numero_ligne in sous_liste:
                numero_trouve = True
                break  # Arrête la boucle une fois que le numéro est trouvé

            # Vérifier si le numéro de ligne entré est présent dans les vecteurs des lignes correspondantes
        if numero_trouve:
            resultat_debut=0
            resultat_fin=0
            if not tableau_debut.empty :
                tableau_debut = tableau_debut[tableau_debut['Lignes'].apply(lambda x: numero_ligne in x)]
                resultat_debut = tableau_debut['Nombre de personnes par ligne'].sum()
            if not tableau_fin.empty :
                    tableau_fin = tableau_fin[tableau_fin['Lignes'].apply(lambda x: numero_ligne in x)]
                    resultat_fin = tableau_fin['Nombre de personnes par ligne'].sum()
            resultat=resultat_debut+resultat_fin
            nb_ligne=round(resultat)
            print(f"Le nombre de personnes supplémentaires sur la plage horaire de l'horaire entré  sur la ligne en raison des JO  est {nb_ligne}.")

            resultat_debut_2=0
            resultat_fin_2=0

            if not tableau_debut.empty:
                tableau_provisoire =tableaux[cle_tableau]
                ligne_provisoire = tableau_provisoire[tableau_provisoire['Station'].str.contains(station)]
                valeur = ligne_provisoire['Part du trafic de la station par rapport à toute la ligne'].values[0]
                tableau_debut['Pourcentage fréquentation'] = valeur
                resultat_debut_2=valeur*resultat_debut
            
            if not tableau_fin.empty:
                tableau_provisoire =tableaux[cle_tableau]
                ligne_provisoire = tableau_provisoire[tableau_provisoire['Station'].str.contains(station)]
                valeur = ligne_provisoire['Part du trafic de la station par rapport à toute la ligne'].values[0]
                tableau_fin['Pourcentage fréquentation'] = valeur
                resultat_fin_2=valeur*resultat_debut
            
            resultat2=resultat_debut_2+resultat_fin_2
            nb_station=round(resultat2)
            print(f"Le nombre de personnes supplémentaires sur la plage horaire de l'horaire entré à la station en raison des JO  est {nb_station}.")

        else:
            print(0)
    else:
        print(0)

In [ ]:
calculer_personnes_supplementaires('2024-07-28 09:20', '1', 'bastille')

Le nombre de personnes supplémentaires sur la ligne en raison des JO  est 17000.
Le nombre de personnes supplémentaires sur à la station en raison des JO  est 984.


In [ ]:
# date_horaire_entree = input("Entrez la date et l'horaire (format YYYY-MM-JJ HH:MM) : ")
# numero_ligne = input("Entrez le numéro de la ligne : ")